In [1]:
import importlib
import sys
import torch
import pickle
import os

sys.path.insert(0, '..')
sys.path.insert(0, '../..')
sys.path.insert(0, '../../..')
sys.path.insert(0, '../../../..')

from model.dropout_uncertainty_enc_dec_LSTM.dropout_uncertainty_model import DropoutUncertaintyEncoderDecoderLSTM

In [2]:
#load model
file_path_model = '../../../training_variational_dropout_log_normal/Helpdesk/Helpdesk_full_grad_lognorm_proactive_conf_check_v1.pkl'
model = DropoutUncertaintyEncoderDecoderLSTM.load(file_path_model, dropout=0.1)

# Load the dataset
file_path_data_set = '../../../../../encoded_data/Helpdesk_proactive/helpdesk_all_lognormal_5_val.pkl'
helpdesk_test_dataset = torch.load(file_path_data_set, weights_only=False)

Data set categories:  ([('Activity', 16, {'Assign seriousness': 1, 'Closed': 2, 'Create SW anomaly': 3, 'DUPLICATE': 4, 'EOS': 5, 'INVALID': 6, 'Insert ticket': 7, 'RESOLVED': 8, 'Require upgrade': 9, 'Resolve SW anomaly': 10, 'Resolve ticket': 11, 'Schedule intervention': 12, 'Take in charge ticket': 13, 'VERIFIED': 14, 'Wait': 15}), ('Resource', 24, {'EOS': 1, 'Value 1': 2, 'Value 10': 3, 'Value 11': 4, 'Value 12': 5, 'Value 13': 6, 'Value 14': 7, 'Value 15': 8, 'Value 16': 9, 'Value 17': 10, 'Value 18': 11, 'Value 19': 12, 'Value 2': 13, 'Value 20': 14, 'Value 21': 15, 'Value 22': 16, 'Value 3': 17, 'Value 4': 18, 'Value 5': 19, 'Value 6': 20, 'Value 7': 21, 'Value 8': 22, 'Value 9': 23}), ('VariantIndex', 170, {'1.0': 1, '10.0': 2, '100.0': 3, '104.0': 4, '107.0': 5, '109.0': 6, '11.0': 7, '110.0': 8, '112.0': 9, '113.0': 10, '114.0': 11, '115.0': 12, '117.0': 13, '118.0': 14, '12.0': 15, '120.0': 16, '122.0': 17, '123.0': 18, '124.0': 19, '125.0': 20, '129.0': 21, '13.0': 22, '130

/home/PSPLab/.local/share/virtualenvs/probabilistic_suffix_prediction_dev-Otdb6r1k/lib/python3.12/site-packages/sklearn/base.py:442: InconsistentVersionWarning: Trying to unpickle estimator OrdinalEncoder from version 1.6.1 when using version 1.7.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/PSPLab/.local/share/virtualenvs/probabilistic_suffix_prediction_dev-Otdb6r1k/lib/python3.12/site-packages/sklearn/base.py:442: InconsistentVersionWarning: Trying to unpickle estimator SimpleImputer from version 1.6.1 when using version 1.7.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/PSPLab/.local/share/virtualenvs/probabilistic_suffix_prediction_d

In [3]:
import evaluation.probabilistic_evaluation
importlib.reload(evaluation.probabilistic_evaluation)
from evaluation.probabilistic_evaluation import ProbabilisticEvaluation

new_eval = ProbabilisticEvaluation(model, helpdesk_test_dataset,
                                   concept_name='Activity',
                                   num_processes=16,
                                   #growing_num_values = [],
                                   growing_num_values = ['case_elapsed_time'],
                                   samples_per_case = 1000,
                                   sample_argmax = False,
                                   use_variance_cat = True,
                                   use_variance_num = True,
                                   all_cat=['Activity'],
                                   all_num=['case_elapsed_time', 'event_elapsed_time'])

In [4]:
!ls ../../../../../../../data/Helpdesk/proact_conf_check/eval_validation_set/

In [5]:
def save_chunk(results, i):
    chunk_number = (i + 1)
    filename = os.path.join(output_dir, f'results_part_{chunk_number:03d}.pkl')
    with open(filename, 'wb') as f:
        pickle.dump(results, f)
    print(f"Saved {len(results)} results to {filename}")

output_dir = '../../../../../../../data/Helpdesk/proact_conf_check/eval_validation_set/'

save_every = 50
results = {}
for i, (case_name, prefix_len, prefix, predicted_suffixes, suffix, mean_prediction) in enumerate(new_eval.evaluate_multi_processing(random_order=True, include_model_states=False)):
    # print(case_name, prefix_len)
    assert((case_name, prefix_len) not in results)
    results[(case_name, prefix_len)] = (prefix, suffix, mean_prediction, predicted_suffixes)
    # print(prefix_len, len(suffix))
    if (i + 1) % save_every == 0:
        save_chunk(results, i)
        results = {}

if len(results):
    save_chunk(results, i)

  0%|          | 0/916 [00:00<?, ?it/s]

Saved 50 results to ../../../../../../../data/Helpdesk/proact_conf_check/eval_validation_set/results_part_050.pkl
Saved 50 results to ../../../../../../../data/Helpdesk/proact_conf_check/eval_validation_set/results_part_100.pkl
Saved 50 results to ../../../../../../../data/Helpdesk/proact_conf_check/eval_validation_set/results_part_150.pkl
Saved 50 results to ../../../../../../../data/Helpdesk/proact_conf_check/eval_validation_set/results_part_200.pkl
Saved 50 results to ../../../../../../../data/Helpdesk/proact_conf_check/eval_validation_set/results_part_250.pkl
Saved 50 results to ../../../../../../../data/Helpdesk/proact_conf_check/eval_validation_set/results_part_300.pkl
Saved 50 results to ../../../../../../../data/Helpdesk/proact_conf_check/eval_validation_set/results_part_350.pkl
Saved 50 results to ../../../../../../../data/Helpdesk/proact_conf_check/eval_validation_set/results_part_400.pkl
Saved 50 results to ../../../../../../../data/Helpdesk/proact_conf_check/eval_validation